# Anomaly Detection using SAR

In this notebook, we explore a **method for identifying anomalies** in Sentinel-1 SAR data available within the Descartes Labs platform that leverages `Workflows` interactive maps.

This example demonstrates the **detection of oil slicks in the ocean** using Sentinel-1 data.  

The cells in this notebook can be run one at a time by using `Shift-Enter`

## Background

Synthetic Aperture Radar (SAR) is an imaging technique that measures the scattering of electromagnetic waves off of a medium. While both the source signal and the reflections are polarized, in this instance we are only going to be looking at monopole images (Vertical transmission/Vertical reflection).

Oil on the water's surface leads to a damping of wave motion, which in turn leads to a low value in the SAR image compared to the background ocean. It is these low values for which we will look in this notebook. Note, however, that simply thresholding the image (as we are doing) will lead to false positives when waters are naturally still. These regions tend to be very large compared to oil slicks.

In this notebook, we will focus on two main causes of oil slicks: subsurface seeps and spills. Seeps happen when underground oil makes its way to the ocean floor and leaks out. This naturally occuring process provides useful information for petroleum exploration, as one of the main risks of drilling a well is determining whether or not there is oil underground (if it's leaking to the surface it exists underground). Spills happen for a variety of reasons, including ship pollution, shipwrecks, and industrial incidents. All of these different types of slicks express differently in SAR images, as shown in this notebook.

## How to use this notebook

Using the drop down menu will populate the date fields and move the map to the chosen AOI. In the background, `Workflows` will be creating a set of SAR images between the two dates chosen, with the `Group days` parameter controlling how many days will be stacked together to generate one image. Clicking `Previous Scene` or `Next Scene` will move from one image to the next. To avoid possibly long computations, after changing the `Start date` or `End date` you will need to click the `Recompute Layers` button.

Be careful making the `Group days` parameter too large. While creating composite images is a very useful technique when working over land, the movement of the ocean makes it much less useful to combine images from different days. Trying to create a monthly average, for example, will simply wash out any signal that may exist during that month.

Experiment with different values of the "Spill Clip Value" parameter to see what will identify a given spill. Notice that the values are very small - SAR signals over the ocean are much weaker than over land, so we need to deal with a very limited range to identify the slicks. Also notice that it is very difficult to capture a slick without also getting large areas of open ocean.

## Areas to investigate

Here is a list of some interesting areas to focus on.

 | Area | Location (Lat, Lon) | Dates | Description |
 | -------------- | --------------------------- | ------------- | -------------------------------------------- |
 | Santa Barbara Channel | 34.378366, -119.813161 | consistently visible | This area contains many natural seeps that are visible with SAR data (and are also responsible for the "tar balls" that show up on the beaches in the area). |
 | Gulf of Mexico | 28.935833, -88.97 | visible intermittently until mid-2019 | Subsurface oil is leaking from a spill caused by a hurricane in 2004. |
 | Kuwait | 28.563148, 48.459282 | 2017-08-10 | A facility in Kuwait spilled ~35,000 gallons of oil into the Persian Gulf. Residual slicks are visible on the 2017-08-14 image as well. |
 | Caspian Sea | 40.0644, 51.3020 | 2017-12-09 | The Oil Rocks complex leaked, releasing ~380,000 gallons in two months. |
 | Mediterranean crash | 43.238734, 9.483196 | 2018-10-07 | A cargo ship crash led to a leak from one of the ships. The ships themselves are visible as bright dots in the image, and the oil is the dark "tail" coming from the ships.|
 | Indonesia | 5.304552, 98.093229 | 2019-11-25 to 2020-02-20 | This looks like a shipwreck - notice the long tail, indicating more oil than would be expected from a natural seep. It is currently unknown what is causing this leak.|

## Import packages

In [ ]:
# Keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [ ]:
# Import packages
import descarteslabs as dl
import descarteslabs.workflows as wf
import ipyleaflet
from utils import Slickfinder_Map

## Creating a map widget

After running this cell, `right-click` on the output and click `Create New View for Output`. This will allow you to make the map fullscreen.

In [ ]:
map_widget = wf.map
map_widget

In [ ]:
#Creating the Slickfinder Map.
# map_widget - wf.map object created above.
# area - one of 'GoM', 'SB', 'Caspian', 'Kuwait', 'Med', or 'Indonesia'
# false_color - Plot the vv data as a single band, or view as a false color composite (min, mean, max mapped to RGB)
# scales, colormap - Default visualization. Colormap cannot be used if false_color=True

Scales_FC = [[0,0.1]]*3
Scales_1Band = [0,0.1]

plot_map = Slickfinder_Map(map_widget, area='SB', false_color=False, scales=Scales_1Band, colormap='gray')

In [ ]:
# If necessary, running the following two lines will clear the map.

# map_widget.clear_controls()
# map_widget.clear_layers()